In [44]:
class EinsteinRiddleSolver:
    def __init__(self):
        # Initialization code
        self.formula = []
        self.colors = {color: [i for i in range(1, 6)] for color in ['red', 'green', 'yellow', 'blue', 'ivory']}
        self.nationalities = {nationality: [i + 5 for i in range(1, 6)] for nationality in ['English', 'Spanish', 'Ukrainian', 'Norwegian', 'Japanese']}
        self.drinks = {drink: [i + 10 for i in range(1, 6)] for drink in ['coffee', 'tea', 'milk', 'orange_juice', 'water']}
        self.cigarettes = {cigarette: [i + 15 for i in range(1, 6)] for cigarette in ['Old_Gold', 'Kools', 'Chesterfields', 'Lucky_Strike', 'Parliaments']}
        self.pets = {pet: [i + 20 for i in range(1, 6)] for pet in ['dog', 'snails', 'fox', 'horse', 'zebra']}
        self.num_vars = 25

        # Add constraints for unique assignments and encode clues
        self.add_unique_constraints()
        self.encode_clues()

    def add_unique_constraints(self):
        # Same logic as before
        for attribute_set in [self.colors, self.nationalities, self.drinks, self.cigarettes, self.pets]:
            for attribute, variables in attribute_set.items():
                self.formula.append(variables)
                for i in range(len(variables)):
                    for j in range(i + 1, len(variables)):
                        self.formula.append([-variables[i], -variables[j]])

    def encode_clues(self):
        # Clue 1: The Englishman lives in the red house.
        for i in range(5):
            english_var = self.nationalities['English'][i]
            red_var = self.colors['red'][i]
            self.formula.append([english_var, -red_var])
            self.formula.append([-english_var, red_var])

        # Clue 2: The Spaniard owns the dog.
        for i in range(5):
            spanish_var = self.nationalities['Spanish'][i]
            dog_var = self.pets['dog'][i]
            self.formula.append([spanish_var, -dog_var])
            self.formula.append([-spanish_var, dog_var])

        # Clue 3: Coffee is drunk in the green house.
        for i in range(5):
            coffee_var = self.drinks['coffee'][i]
            green_var = self.colors['green'][i]
            self.formula.append([coffee_var, -green_var])
            self.formula.append([-coffee_var, green_var])

# Create an instance of the class to verify constraints
solver = EinsteinRiddleSolver()
solver.formula[-10:] # Showing the last 10 constraints, including the ones for Clue 3


[[11, -1],
 [-11, 1],
 [12, -2],
 [-12, 2],
 [13, -3],
 [-13, 3],
 [14, -4],
 [-14, 4],
 [15, -5],
 [-15, 5]]

In [34]:
# Clue 1: The Englishman lives in the red house.
for house in houses:
    KG.add_edge(house, 'Englishman', color='Red')

# Clue 2: The Spaniard owns the dog.
for house in houses:
    KG.add_edge(house, 'Spaniard', pet='Dog')
    # Clue 3: Coffee is drunk in the green house.
for house in houses:
    KG.add_edge(house, 'Coffee', color='Green')

# Clue 4: The Ukrainian drinks tea.
for house in houses:
    KG.add_edge(house, 'Ukrainian', drink='Tea')

# Clue 5: The green house is immediately to the right of the ivory house.
for i in range(4):
    KG.add_edge(houses[i], houses[i + 1], color_from='Ivory', color_to='Green')

# Clue 6: The Old Gold smoker owns snails.
for house in houses:
    KG.add_edge(house, 'Old Gold', pet='Snails')

# Clue 7: Kools are smoked in the yellow house.
for house in houses:
    KG.add_edge(house, 'Kools', color='Yellow')

# Clue 8: Milk is drunk in the middle house.
KG.add_edge(houses[2], 'Milk')

# Clue 9: The Norwegian lives in the first house.
KG.add_edge(houses[0], 'Norwegian')

# Clue 10: The man who smokes Chesterfields lives in the house next to the man with the fox.
for i in range(4):
    KG.add_edge(houses[i], houses[i + 1], cigarette='Chesterfields', pet='Fox')
    KG.add_edge(houses[i + 1], houses[i], cigarette='Chesterfields', pet='Fox')

# Clue 11: Kools are smoked in the house next to the house where the horse is kept.
for i in range(4):
    KG.add_edge(houses[i], houses[i + 1], cigarette='Kools', pet='Horse')
    KG.add_edge(houses[i + 1], houses[i], cigarette='Kools', pet='Horse')

# Clue 12: The Lucky Strike smoker drinks orange juice.
for house in houses:
    KG.add_edge(house, 'Lucky Strike', drink='Orange Juice')

# Clue 13: The Japanese smokes Parliaments.
for house in houses:
    KG.add_edge(house, 'Japanese', cigarette='Parliaments')

# Clue 14: The Norwegian lives next to the blue house.
KG.add_edge(houses[0], houses[1], color='Blue')
KG.add_edge(houses[1], houses[0], nationality='Norwegian')


In [35]:
pos = nx.spring_layout(KG)
nx.set_node_attributes(KG, pos, 'pos')
node_x = [KG.nodes[node]['pos'][0] for node in KG.nodes()]
node_y = [KG.nodes[node]['pos'][1] for node in KG.nodes()]

In [37]:
# Convert to Plotly format
edge_x = []
edge_y = []
for edge in KG.edges():
    x0, y0 = KG.nodes[edge[0]]['pos']
    x1, y1 = KG.nodes[edge[1]]['pos']
    edge_x.append(x0)
    edge_y.append(y0)
    edge_x.append(x1)
    edge_y.append(y1)

# Create a scatter plot for edges
edge_trace = go.Scatter(
    x=edge_x, y=edge_y,
    line=dict(width=0.5, color='#888'),
    hoverinfo='none',
    mode='lines')

# Create a scatter plot for nodes
node_trace = go.Scatter(
    x=node_x, y=node_y,
    mode='markers',
    hoverinfo='text',
    marker=dict(showscale=True, colorscale='YlGnBu', size=10))

# Layout for the plot
layout = go.Layout(
    showlegend=False,
    hovermode='closest',
    margin=dict(b=0, l=0, r=0, t=0),
    xaxis=dict(showgrid=False, zeroline=False),
    yaxis=dict(showgrid=False, zeroline=False))

# Create the figure
fig = go.Figure(data=[edge_trace, node_trace], layout=layout)
fig.show()
